# `waterSpec` Demo

This notebook provides a detailed walkthrough of the `waterSpec` package. We will load a sample time series, preprocess it, perform spectral analysis, fit the spectrum, interpret the results, and visualize the output.

## 1. Import the package

In [ ]:


import waterSpec as ws

## 2. Load and Inspect Data

The `load_data` function can handle `.csv`, `.json`, and `.xlsx` files.

In [ ]:
csv_file_path = 'sample_data.csv'
time, concentration_series = ws.load_data(csv_file_path, time_col='timestamp', data_col='concentration')

print(f"Loaded {len(time)} data points from {csv_file_path}.")
print("Data type:", type(concentration_series))

## 3. Preprocess the Data

### 3a. Handling Censored Data

Water quality data often contains censored values (e.g., below a detection limit, `<5.0`). Our package can handle this. Let's load a dataset with censored values.

In [ ]:
censored_file_path = 'censored_data.csv'
_, censored_series = ws.load_data(censored_file_path, time_col='timestamp', data_col='concentration')
print("Original censored data:")
print(censored_series)

In [ ]:
# Handle censored data using the 'multiplier' strategy
numeric_concentration = ws.handle_censored_data(
    censored_series,
    strategy='multiplier',
    lower_multiplier=0.5,
    upper_multiplier=1.1
)

print("Data after handling censoring:")
print(numeric_concentration)

### 3b. Detrending

Now we can detrend the clean, numeric data.

In [ ]:
# For the rest of the analysis, we'll use the original, non-censored data.
numeric_concentration = pd.to_numeric(concentration_series)
detrended_concentration = ws.detrend(numeric_concentration)

## 4. Perform Spectral Analysis

Now we calculate the Lomb-Scargle periodogram.

In [ ]:
frequency, power = ws.calculate_periodogram(time, detrended_concentration)

## 5. Fit the Spectrum and Interpret the Results

In [ ]:
# We use the bootstrap function to get confidence intervals
fit_results = ws.fit_spectrum_with_bootstrap(frequency, power, n_bootstraps=500)
beta = fit_results['beta']
interpretation = ws.interpret_beta(beta)

print(f"Spectral Exponent (β): {beta:.2f}")
print(f"95% CI: [{fit_results['beta_ci_lower']:.2f}, {fit_results['beta_ci_upper']:.2f}]")
print(f"R-squared of fit: {fit_results['r_squared']:.2f}")
print(f"Interpretation: {interpretation}")

## 6. Visualize the Spectrum

Finally, we can plot the power spectrum and the fitted line.

In [ ]:
# The `plot_spectrum` function can show the plot directly.
# We set `show=True` to see it here.
ws.plot_spectrum(frequency, power, fit_results=fit_results, show=True)

## 7. Simplified Workflow with `run_analysis`

All the steps above can be performed in a single function call using the high-level `run_analysis` function. This is the recommended approach for most use cases.

In [ ]:
import pprint

results = ws.run_analysis(
    file_path='sample_data.csv',
    time_col='timestamp',
    data_col='concentration',
    do_plot=True,
    n_bootstraps=500
)

print("--- Full Analysis Results ---")
pprint.pprint(results)

## 8. Advanced: Segmented Regression for Multifractal Analysis

If you suspect that the scaling behavior of your time series changes at different frequencies, you can use segmented regression to find breakpoints.

In [ ]:
# Run the analysis with analysis_type='segmented'
segmented_results = ws.run_analysis(
    file_path='sample_data.csv', # Using simple data for demonstration
    time_col='timestamp',
    data_col='concentration',
    analysis_type='segmented',
    do_plot=False # Plotting for segmented regression is not yet customized
)

print("--- Segmented Analysis Results ---")
pprint.pprint(segmented_results)

## 9. Advanced: Non-Linear Detrending

If your data has a non-linear trend, you can use LOESS detrending.

In [ ]:
# Run the analysis with detrend_method='loess'
loess_results = ws.run_analysis(
    file_path='sample_data.csv', # Using simple data for demonstration
    time_col='timestamp',
    data_col='concentration',
    detrend_method='loess'
)

print("--- LOESS Detrended Analysis Results ---")
pprint.pprint(loess_results)